In [22]:
import streamlit as st
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import json
from sklearn.metrics.pairwise import cosine_similarity
from gemini import rerank_with_gemini
from openaimodel import rerank_with_openai

# Load data
items = pd.read_csv("cleaned_items_with_metadata.csv")
item_texts = items["full_text"].tolist()
queries = pd.read_csv("./data/queries.csv")["search_term_pt"].tolist()
item_embeddings = np.load("./embeddings/text_embeddings_openai_small.npy")
query_embeddings = np.load("./embeddings/query_embeddings_openai_small.npy")
 # Your precomputed 768-dim vector

similarity_matrix = cosine_similarity(query_embeddings, item_embeddings)
top_k = 8
top_k_indices = np.argsort(-similarity_matrix, axis=1)[:, :top_k]

In [23]:
results = []

for i, item_idxs in enumerate(top_k_indices):
    query_text = queries[i]
    matched_items = [item_texts[j] for j in item_idxs]
    results.append({
        "query": query_text,
        "top_k_results": matched_items
    })

In [ ]:
from openai import OpenAI

# Configure the Gemini client (OpenAI-compatible)
client = OpenAI(api_key="XXXXXXXXXXXXX")


def evaluate_top3_with_llm(query, top3_items, model="gemini-1.5-pro-latest"):
    """
    Evaluates how relevant each item in top-3 is to the query using Gemini.
    Returns 3 float scores (0–2) in original order.
    """

    prompt = f"""
You are a food search evaluator.

Given a search query and three food item descriptions, rate how relevant each item is to the query on a scale from 0 to 2:
- 2: Highly relevant (perfect match or very close)
- 1: Somewhat relevant (partially matches or related)
- 0: Not relevant (not matching or off-topic)

Only output the scores as a comma-separated list, in order. No explanation.

Query: "{query}"

Items:
1. {top3_items[0]}
2. {top3_items[1]}
3. {top3_items[2]}

Output format: 2, 1, 0
"""

    response = client.responses.create(
        model="gpt-4.1",
        input=prompt
    )

    return response.output[0].content[0].text.strip()


In [24]:
index = 5

response = evaluate_top3_with_llm(queries[index], results[index]["top_k_results"])
print(f"Response: {response}") 

Response: 0, 1, 1


In [20]:
# get all 3 scores separtely and sum up on total

scores = list(map(float, response.split(',')))
total_n1 = scores[0]
total_n2 = scores[1]
total_n3 = scores[2]
print(f"Total scores: {total_n1}, {total_n2}, {total_n3}")

Total scores: 0.0, 1.0, 1.0


## Evaluation without rerank

In [21]:
from tqdm import tqdm

total_n1 = 0
total_n2 = 0
total_n3 = 0
for i, result in tqdm(enumerate(results), total=len(results), desc="Evaluating queries"):
    query = result["query"]
    top3_items = result["top_k_results"]
    
    # Evaluate with LLM
    response = evaluate_top3_with_llm(query, top3_items)
    scores = list(map(float, response.split(',')))
    
    total_n1 += scores[0]
    total_n2 += scores[1]
    total_n3 += scores[2]
    # Print results

print(f"Query: {query}")
print(f"Top 3 items: {top3_items}")
print(f"Scores: {scores}")
print(f"Total scores: {total_n1}, {total_n2}, {total_n3}")

avg_n1 = total_n1 / len(results)
avg_n2 = total_n2 / len(results)
avg_n3 = total_n3 / len(results)


Evaluating queries: 100%|██████████| 100/100 [03:36<00:00,  2.17s/it]

Query: Lanche reconfortante para a madrugada
Top 3 items: ["Lanche dona lurdes. Lanche de kafta com queijo, rucula maionese de alho e azeitonas pretas. Categoria: Lanches boquinha de anjo. Taxonomia: {'l0': 'ALIMENTOS_PREPARADOS', 'l1': 'SANDUICHES', 'l2': 'HAMBURGUERES'}", "Lanche de Linguiça Especial. Desfrute do nosso 'Lanche de Linguiça Especial', uma combinação perfeitamente harmonizada de linguiça saborosamente aberta, alface fresca e crocante, vinagrete suculento e apetitoso, cream cheese cremoso e indulgente, mussarela derretida e aconchegante, tudo envolvido em sua escolha de um pão francês crocante e dourado ou um pão de hambúrguer macio e esponjoso. Esta é a refeição perfeita para satisfazer a sua fome e despertar os seus sentidos. Uma delícia da categoria 'Lanches' que promete uma explosão de sabor a cada mordida!. Categoria: Lanches. Taxonomia: {'l0': 'ALIMENTOS_PREPARADOS', 'l1': 'SANDUICHES', 'l2': 'HAMBURGUERES'}", "Bebida Láctea Uht Chocolate com Café Frappuccino Starb

## Evaluation for rerank

In [ ]:
from tqdm import tqdm
from utils.openaimodel import rerank_with_openai

total_n1_rerank = 0
total_n2_rerank = 0
total_n3_rerank = 0
for i, result in tqdm(enumerate(results), total=len(results), desc="Evaluating reranked queries"):
    query = result["query"]
    top3_items = result["top_k_results"]
    new_rank = rerank_with_openai(query, top3_items)
    reranked_items = [top3_items[int(i)-1] for i in new_rank.split(",")]
    
    # Evaluate with LLM
    response = evaluate_top3_with_llm(query, reranked_items)
    
    scores = list(map(float, response.split(',')))
    
    total_n1_rerank += scores[0]
    total_n2_rerank += scores[1]
    total_n3_rerank += scores[2]

print(f"Query example: {query}")
print(f"Reranked items example: {reranked_items}")
print(f"Scores example: {scores}")
print(f"Total reranked scores: {total_n1_rerank}, {total_n2_rerank}, {total_n3_rerank}")

avg_n1_rerank = total_n1_rerank / len(results)
avg_n2_rerank = total_n2_rerank / len(results)
avg_n3_rerank = total_n3_rerank / len(results)

print(f"Average reranked scores: {avg_n1_rerank:.2f}, {avg_n2_rerank:.2f}, {avg_n3_rerank:.2f}")
print(f"Baseline average scores: {avg_n1:.2f}, {avg_n2:.2f}, {avg_n3:.2f}")


Evaluating reranked queries: 100%|██████████| 100/100 [05:24<00:00,  3.24s/it]

Query example: Lanche reconfortante para a madrugada
Reranked items example: ["Lanche de Linguiça Especial. Desfrute do nosso 'Lanche de Linguiça Especial', uma combinação perfeitamente harmonizada de linguiça saborosamente aberta, alface fresca e crocante, vinagrete suculento e apetitoso, cream cheese cremoso e indulgente, mussarela derretida e aconchegante, tudo envolvido em sua escolha de um pão francês crocante e dourado ou um pão de hambúrguer macio e esponjoso. Esta é a refeição perfeita para satisfazer a sua fome e despertar os seus sentidos. Uma delícia da categoria 'Lanches' que promete uma explosão de sabor a cada mordida!. Categoria: Lanches. Taxonomia: {'l0': 'ALIMENTOS_PREPARADOS', 'l1': 'SANDUICHES', 'l2': 'HAMBURGUERES'}", "Lanche dona lurdes. Lanche de kafta com queijo, rucula maionese de alho e azeitonas pretas. Categoria: Lanches boquinha de anjo. Taxonomia: {'l0': 'ALIMENTOS_PREPARADOS', 'l1': 'SANDUICHES', 'l2': 'HAMBURGUERES'}", "Copo da Felicidade Morango com Ninh